In [1]:
import pandas as pd
import numpy as np

from lightgbm import LGBMRegressor
import gresearch_crypto

In [2]:
TRAIN_CSV = '../input/g-research-crypto-forecasting/train.csv'
ASSET_DETAILS_CSV = '../input/g-research-crypto-forecasting/asset_details.csv'

In [3]:
df_train = pd.read_csv(TRAIN_CSV)
df_train.head()

In [4]:
df_asset_details = pd.read_csv(ASSET_DETAILS_CSV)
df_asset_details.head()

In [5]:
def upper_shadow(df):
    return df['High'] - np.maximum(df['Close'], df['Open'])

def lower_shadow(df):
    return np.minimum(df['Close'], df['Open']) - df['Low']

def get_features(df):
    df_feat = df[['Count', 'Open', 'High', 'Low', 'Close', 'Volume', 'VWAP']].copy()
    df_feat['Upper_Shadow'] = upper_shadow(df_feat)
    df_feat['Lower_Shadow'] = lower_shadow(df_feat)
    return df_feat

def get_Xymodel_for_asset(df_train,asset_id):
    df = df_train[df_train["Asset_ID"]==asset_id]

    df_proc = get_features(df)
    df_proc['y'] = df['Target']
    df_proc = df_proc.dropna(how="any")
    
    X = df_proc.drop('y', axis=1)
    y = df_proc['y']
    
    model = LGBMRegressor(n_estimators=1500, num_leaves=700, learning_rate = 0.09)
    model.fit(X,y)
    return X, y, model

In [6]:
Xs = {}
ys = {}

models = {}

for asset_id, asset_name in zip(df_asset_details['Asset_ID'], df_asset_details['Asset_Name']):
    print(f"Training model for {asset_name:<16}(ID={asset_id:<2})")
    X, y, model = get_Xymodel_for_asset(df_train, asset_id)
    Xs[asset_id], ys[asset_id], models[asset_id] = X, y, model

In [7]:
x = get_features(df_train.iloc[1])
y_pred = models[0].predict([x])
y_pred = 0


In [11]:
import pdb; 
#pdb.set_trace();

In [12]:
env = gresearch_cryto.make_env()
iter_test = env.iter_test()


for i, (df_test, df_pred) in enumerate(iter_test):
    for j, row in df_test.iterrows():
        model = models[row['Asset_ID']]
        x_test = get_features(row)
        y_pred = model.predict([x_test])[0]
        
        df_pred.loc[df_pred['row_id'] == row['row_id'], 'Target'] = y_pred
    
    env.predict(df_pred)

In [ ]:
df_pred.to_csv('submission.csv',index=False)

In [ ]:
df_test = pd.concat(all_df_test)
df_test['datetime'] = pd.to_datetime(df_test['timestamp'], unit='s')
df_train['datetime'] = pd.to_datetime(df_train['timestamp'], unit='s')

In [ ]:
df_train['datetime'].max()

In [ ]:
df_test['datetime'].min()